# Consolidando MDT

In [1]:
import pandas as pd
import numpy as np
import gc

pd.set_option('display.max_columns',None)

### Cargando Poblacion

In [2]:
### Cargando Base
poblacion_total = pd.read_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_pilotoAgosto",engine='pyarrow')

In [3]:
poblacion_total['periodo'] = '08-2020'

In [4]:
poblacion_total['periodo'].value_counts().sort_index()

08-2020    342602
Name: periodo, dtype: int64

In [5]:
## TODO: POR AHORA SOLO INFO 2019
#poblacion_total = poblacion_total[poblacion_total['mes_campaña']<'2020-01-01']

In [5]:
#Creación de fecha correspondiente para cruzar con la base buró
poblacion_total['data_camp']=np.where((poblacion_total['periodo']=='08-2020') ,202006,
                                      199901)

In [6]:
#Creación de fecha correspondiente para cruzar con la base buró
poblacion_total['fecha_buro']=np.where((poblacion_total['periodo']=='08-2020') ,202004,
                                       199901)

In [7]:
#Creación de fecha correspondiente para cruzar con la base activo
poblacion_total['fecha_activo']=np.where((poblacion_total['periodo']=='08-2020') ,202006,
                                         199901)

In [8]:
#Creación de fecha correspondiente para cruzar con la base pasivo
poblacion_total['fecha_pasivo']=np.where((poblacion_total['periodo']=='08-2020') ,202006,
                                         199901)

In [9]:
#Creación de fecha correspondiente para cruzar con la base libranza
poblacion_total['fecha_lib']=np.where((poblacion_total['periodo']=='08-2020') ,202007,
                                         199901)

In [10]:
#Creación de fecha correspondiente para cruzar con la base tx activo
poblacion_total['fecha_tx_act']=np.where((poblacion_total['periodo']=='08-2020') ,202007,
                                         199901)

In [11]:
#Creación de fecha correspondiente para cruzar con la base tx pasivo
poblacion_total['fecha_tx_pav']=np.where((poblacion_total['periodo']=='08-2020') ,202006,
                                         199901)

In [12]:
poblacion_total

,id_cliente,grupo,grupo_piloto,periodo,data_camp,fecha_buro,fecha_activo,fecha_pasivo,fecha_lib,fecha_tx_act,fecha_tx_pav
0,102652303931898101,1,control,08-2020,202006,202004,202006,202006,202007,202007,202006
1,102652295880533801,2,priorizado,08-2020,202006,202004,202006,202006,202007,202007,202006
2,102652296797857801,1,control,08-2020,202006,202004,202006,202006,202007,202007,202006
3,102652296820136301,2,priorizado,08-2020,202006,202004,202006,202006,202007,202007,202006
4,102652298458155701,2,priorizado,08-2020,202006,202004,202006,202006,202007,202007,202006
...,...,...,...,...,...,...,...,...,...,...,...
342597,624859635235423902,2,priorizado,08-2020,202006,202004,202006,202006,202007,202007,202006
342598,133959641475029202,1,control,08-2020,202006,202004,202006,202006,202007,202007,202006
342599,703659639027269402,1,control,08-2020,202006,202004,202006,202006,202007,202007,202006
342600,218159639017533402,1,control,08-2020,202006,202004,202006,202006,202007,202007,202006


In [14]:
# tx activos t-1
# tx pasivos t-1

In [13]:
# Cast ID
poblacion_total['id_cliente']=poblacion_total['id_cliente'].astype(int)

In [14]:
poblacion_total = poblacion_total.drop_duplicates()

### Guardando base pob

In [15]:
poblacion_total.shape

(342602, 11)

In [16]:
### Guardando Base:
path_final = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/base_poblacionPilotoAg'

poblacion_total.to_parquet(path_final, engine = 'pyarrow', index = False)

### Base tx pasv

In [26]:
import s3fs
import pyarrow.parquet as pq

fs = s3fs.S3FileSystem()

#input_path = "s3://adl-refined-dev-popular/parquet/TC_adquisicion/tx_pasivo_202006_vars_finales_model"
input_path = "s3://adl-refined-dev-popular/parquet/TC_adquisicion/total_tdc_piloto_sep"

input = input_path
dataset = pq.ParquetDataset(input, filesystem=fs)
lib = dataset.read()
lib = lib.to_pandas()

del dataset

In [27]:
lib.count().head(50)

id_cliente                        342602
fecha_tx_pav                      342602
fecha_pasivo                      342602
fecha_lib                         342602
fecha_activo                      342602
fecha_buro                        342602
grupo                             342602
grupo_piloto                      342602
data_camp                         342602
fecha_tx_act                      342602
acep_oferta_prev                     205
num_act_utl_meses                    205
num_camp_ult_meses                   205
num_meses_ult_actv                     1
num_meses_ult_camp                   205
num_no_aceptado                      205
prom_cupo_aprob                        0
numero_obligaciones_activasdif    331559
porcentaje_utilizacion            331559
quanto_mod                        331559
valor_cuotas_codeudores_smlv      331559
valor_utilizado_smlv              331559
num_aper_ultim_1meses_LB          335834
num_aper_ultim_1meses_TC          335834
num_aper_ultim_1

In [18]:
lib.head()

,id_cliente,PERIODO,num_tx_ult_1mes_ahr,sum_tx_linea_ahr,dias_desde_ult_tx_ahr,sum_tx_linea,sum_tx_nolinea
0,101958376376088702,202006,1.000000,1.0,28,1.0,NaN
1,102652294123116001,202006,2.333333,NaN,11,NaN,3.0
2,102652294793663401,202006,2.666667,3.0,4,3.0,NaN
3,102652295678115801,202006,1.000000,1.0,4,1.0,NaN
4,102652296335428801,202006,3.000000,5.0,11,5.0,NaN


In [21]:
lib.dtypes

id_cliente                object
PERIODO                   object
num_tx_ult_1mes_ahr      float64
sum_tx_linea_ahr         float64
dias_desde_ult_tx_ahr      int32
sum_tx_linea             float64
sum_tx_nolinea           float64
fecha_tx_pav               int64
dtype: object

In [20]:
lib['fecha_tx_pav'] = lib['PERIODO'].astype('int')

In [22]:
lib.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/tx_pasivo_202006_vars_finales_model", engine = 'pyarrow', index=False)